**DEV**:
* **where to run**: thesis-remote (Workspace) [SSH: login*.pegasus.kl.dfki.de]
* **kernel**: appraise-env (Python 3.10.6)
* **directory**: `~/Appraise`
* Appraise server (`manage.py runserver`) has to be running simultaneously

**PROD**:
* **where to run**: um-server (Workspace) [SSH: um]
* **kernel**: Python 3.8.10
* `python3 manage.py runserver` running simultaneously

In [1]:
import sys
sys.path.append("/home/juliafalcao/")
sys.path.append("/home/juliafalcao/experiments")

from experiments.constants import * 
from experiments.utils import *

from random import randint
from collections import OrderedDict
import json

In [2]:
SRC, TGT = SPANISH, BASQUE
# SRC, TGT = ENGLISH, MALTESE

LP_FOLDER = f"/home/juliafalcao/thesis_data/{SRC}-{TGT}"
# BATCHES_FILENAME = f"batches.{SRC}-{TGT}.json"
BATCHES_FILENAME = f"ref-batches.{SRC}-{TGT}.json"
BATCHES_PATH = f"{LP_FOLDER}/{BATCHES_FILENAME}"
print("lp folder:", LP_FOLDER)
print("batches filename:", BATCHES_FILENAME)

lp folder: /home/juliafalcao/thesis_data/es-eu
batches filename: ref-batches.es-eu.json


## Create batches

In [7]:
# configurations
_task_definition = OrderedDict({
    "TGT": 80,
    "CHK": 0,
    "REF": 10,
    "BAD": 10,
})


In [8]:
TASK_DEFINITION = ":".join(map(str, _task_definition.values()))
SRC_FILE = f"{LP_FOLDER}/src.{SRC}"
REF_FILE = f"{LP_FOLDER}/ref.{TGT}"
SYSTEMS_FOLDER = f"{LP_FOLDER}/systems/"

print("src file:", SRC_FILE)
print("ref file:", REF_FILE)
print("/systems:", os.listdir(SYSTEMS_FOLDER))

src file: /home/juliafalcao/thesis_data/es-eu/src.es
ref file: /home/juliafalcao/thesis_data/es-eu/ref.eu
/systems: []


In [9]:
# call the command from bash but using variables set in Python
! python manage.py CreateDirectAssessmentData \
    100 \
    $SRC.code3 \
    $TGT.code3 \
    $LP_FOLDER/src.$SRC \
    $LP_FOLDER/ref.$TGT \
    $SYSTEMS_FOLDER \
    $BATCHES_PATH \
    --task-definition $TASK_DEFINITION \
    --required-annotations 3 \
    --source-based \
    --all-batches

Using task definition: (80, 0, 10, 10)
Traceback (most recent call last):
  File "manage.py", line 34, in <module>
    execute_from_command_line(sys.argv)
  File "/home/juliafalcao/appraise-env/lib/python3.8/site-packages/django/core/management/__init__.py", line 419, in execute_from_command_line
    utility.execute()
  File "/home/juliafalcao/appraise-env/lib/python3.8/site-packages/django/core/management/__init__.py", line 413, in execute
    self.fetch_command(subcommand).run_from_argv(self.argv)
  File "/home/juliafalcao/appraise-env/lib/python3.8/site-packages/django/core/management/base.py", line 354, in run_from_argv
    self.execute(*args, **cmd_options)
  File "/home/juliafalcao/appraise-env/lib/python3.8/site-packages/django/core/management/base.py", line 398, in execute
    output = self.handle(*args, **options)
  File "/var/www/rival/public_html/translation-eval/EvalData/management/commands/CreateDirectAssessmentData.py", line 285, in handle
    source_file = Command._load_

## Create campaign

In [4]:
# configuration

# CAMPAIGN_NAME = f"{SRC.code3.capitalize()}{TGT.code3.capitalize()}V1"
CAMPAIGN_NAME = "SpaEusV2"
print("campaign name:", CAMPAIGN_NAME)

campaign name: SpaEusV2


In [5]:
manifest = {
    "CAMPAIGN_URL": "http://127.0.0.1:8000/dashboard/sso/",
    "CAMPAIGN_NAME": CAMPAIGN_NAME,
    "CAMPAIGN_KEY": CAMPAIGN_NAME,
    "CAMPAIGN_NO": randint(0,100),
    "REDUNDANCY": 1,

    "TASKS_TO_ANNOTATORS": [
        [ SRC.code3, TGT.code3, "uniform", 1, 1 ]
    ]
}

assert type(manifest["TASKS_TO_ANNOTATORS"]) == list and type(manifest["TASKS_TO_ANNOTATORS"][0]) == list

In [6]:
MANIFEST_PATH = f"{LP_FOLDER}/manifest.json"

with open(MANIFEST_PATH, mode="w+") as f:
    json.dump(manifest, f, indent=4)

In [7]:
! python3 manage.py StartNewCampaign \
    $MANIFEST_PATH \
    --batches-json $BATCHES_PATH

JSON manifest path: '/home/juliafalcao/thesis_data/es-eu/manifest.json'
CSV output path: None
Excel output path: None
No task type found in the manifest file, assuming it is "Direct". If this is incorrect, define "TASK_TYPE" in the manifest file.
### Running InitCampaign
All languages: [('spa', 'eus')]
Identified superuser: falcao
Processed Market/Metadata instances
### Creating a new campaign
- '/home/juliafalcao/thesis_data/es-eu/ref-batches.es-eu.json'
Batch: /home/juliafalcao/thesis_data/es-eu/ref-batches.es-eu.json
  Market: spa_eus_SpaEusV2
  Metadata: spa->eus/SpaEusV2["1.0"]
Uploaded file name: Batches/ref-batches.es-eu.json
Campaign name: SpaEusV2
### Running validatecampaigndata
Campaign name: SpaEusV2
Batch name: Batches/ref-batches.es-eu.json
Validated 1 batches
### Running ProcessCampaignData
Batches/ref-batches.es-eu.json 1
6 ref.eu
137 b'Egileak egiazkotasuna bilatu behar du bere gaia tratatzean... bere pentsakera adierazteko eskubidea izan, bere ideiak bere hitzetan ema